In [1]:
import torus as t
import pandas as pd
import numpy as np
import constants as const
import stellar_spectra as ss
import scipy.integrate as sp_int
from flatstar import draw
import matplotlib.pyplot as plt

 ChiantiPy version 0.15.0 
 found PyQt5 widgets
 using PyQt5 widgets


In [2]:
solar_abundance = pd.read_excel('../SolarSystemAbundances_Lodders2021.xlsx', usecols = [1, 11], names = ['Element', 'Abundance'], skiprows=3, dtype={'Element':'string'}) #in astronomical abundance

#change to abundance relative to hydrogen
solar_abundance['Abundance'] = 10**(solar_abundance['Abundance'] - 12)
sa = dict(zip(solar_abundance['Element'], solar_abundance['Abundance']))
sola =  {k.lower(): v for k, v in sa.items()}
species_keys = ['h', 'he', 'c', 'ca', 'fe', 'na', 'al', 'o', 'mg', 'si', 'n', 'ne', 'ar', 's']
species = dict((k, sola[k]) for k in species_keys)
species_state = ['he_1', 'he_1', 'c_2', 'c_3', 'c_4', 'o_1', 'mg_1', 'mg_2', 'ca_1', 'ca_2', 'fe_2', 'na_1', 'al_2']
lines = [['he_3-2', 'he_3-1', 'he_3-0'], ['he_32-2', 'he_32-1', 'he_32-0'], ['c2_1335'], ['c3_977'], ['c4_1548'], ['o1_1302'], ['mg_2852'], ['mg2_h'], ['ca_422'], ['ca2_h'], ['fe2_2600'], ['na_5890'], ['al2_1670']]
wvls = [np.linspace(1.0830e-4, 1.0836e-4, 100),
        np.linspace(3.8884095227659365e-05, 3.891048491911004e-05, 100),
        np.linspace(1.3340868443910739e-05, 1.3349771556089262e-05, 100),
        np.linspace(9.766940985356417e-06, 9.773459014643584e-06, 100),
        np.linspace(1.5476705761699857e-05, 1.5487034238300142e-05, 100),
        np.linspace(1.3017336399479635e-05, 1.3026023600520364e-05, 100),
        np.linspace(2.851175625037526e-05, 2.8530783749624736e-05, 100), 
        np.linspace(2.7954172303946095e-05, 2.7972827696053907e-05, 100),
        np.linspace(4.225317703872711e-05, 4.228137496127289e-05, 100), 
        np.linspace(3.933464488908903e-05, 3.936089511091097e-05, 100),
        np.linspace(2.5993056685346406e-05, 2.601040331465359e-05, 100),
        np.linspace(5.8879853080489675e-05, 5.8919146919510327e-05, 100),
        np.linspace(1.6702296808766137e-05, 1.6713443191233865e-05, 100)]

In [3]:
K_star = {'mass' : 0.74, 'radius' : 0.77, 'stellar_type' : 'early-K'} 
Sun = {'mass' : 1, 'radius' : 1, 'stellar_type':'solar'}
F_star = {'mass' : 1.19, 'radius' : 1.23, 'stellar_type' : 'late-F'} 


#star
star = F_star

#star grid
flux_map_oot = draw.star(100, supersampling=10).intensity

#disk config
P = 4 * const.day
a = (P**2 * const.G * star['mass'] * const.m_sun / 4 / np.pi**2)**(1/3)
T=np.float64(10**4)
cs = np.sqrt(const.kb * T / (2 * const.mp))
Lcor = 1e6 * P / 4 / np.pi

#spectrum 
star_base_spectrum = ss.stellar_spectra(star['stellar_type'], semi_major_axis = a / (star['radius']*const.r_sun))
star_spectrum = star_base_spectrum.get_rescaled_spectrum(norm_wav=[1.24e-6, 9.12e-6], norm_flux=10**5, rescale_rng=[1e-6, 1.2e-5])


#disk = t.Torus(species=species, stellar_spectrum=star_spectrum, Ms=star['mass']*const.m_sun, Rs=star['radius']*const.r_sun, ap=a, T=T, Rtorus=[a-Lcor, a+Lcor], tau=10)

#get_EW
"""for j, line in enumerate(lines):
    tau_line = disk.get_optical_depth(species_state[j], lines[j], wvls[j][50])
    print(line)
    print(tau_line)
    i_line, tau_line = disk.do_transit(species_state[j], lines[j], wvls[j], flux_map_oot, indexing='image')
    fig1, ax1 = plt.subplots()
    im1 = ax1.imshow(flux_map_oot * np.exp(-tau_line[:, : , 50]), origin='lower')
    fig1.colorbar(im1, ax=ax1)
    plt.show()
    plt.plot(wvls[j], i_line)
    plt.show()"""

"for j, line in enumerate(lines):\n    tau_line = disk.get_optical_depth(species_state[j], lines[j], wvls[j][50])\n    print(line)\n    print(tau_line)\n    i_line, tau_line = disk.do_transit(species_state[j], lines[j], wvls[j], flux_map_oot, indexing='image')\n    fig1, ax1 = plt.subplots()\n    im1 = ax1.imshow(flux_map_oot * np.exp(-tau_line[:, : , 50]), origin='lower')\n    fig1.colorbar(im1, ax=ax1)\n    plt.show()\n    plt.plot(wvls[j], i_line)\n    plt.show()"

In [4]:
#compare K, G and F abundances for same EUV flux

diskK = t.Torus(species=species, stellar_spectrum=star_spectrum, Ms=star['mass']*const.m_sun, Rs=star['radius']*const.r_sun, ap=a, T=T, Rtorus=[a-Lcor, a+Lcor], nH=1e7)





1.7101721796706992


In [7]:
diskK = t.Torus(species=species, stellar_spectrum=star_spectrum, Ms=star['mass']*const.m_sun, Rs=star['radius']*const.r_sun, ap=a, T=T, Rtorus=[a-Lcor, a+Lcor], nH=1e8)

1.7101721796706992


In [8]:
diskK.species_ion_abundances

{'h': array([0.00386038, 0.99613962]),
 'he': array([2.17976137e-02, 4.15881033e-06, 9.46219319e-01, 3.19789084e-02]),
 'c': array([2.28754553e-04, 3.00285067e-01, 6.86882263e-01, 1.25766371e-02,
        2.72778180e-05, 5.29079788e-12, 4.60553137e-19]),
 'ca': array([1.09992495e-004, 3.21770893e-002, 8.21595246e-001, 1.45459113e-001,
        6.58499920e-004, 5.91917013e-008, 1.05628775e-013, 5.81243948e-021,
        1.18695712e-028, 5.22890507e-036, 1.33303010e-043, 1.19050949e-050,
        1.14464564e-058, 1.51946064e-066, 5.49429261e-075, 1.07193959e-083,
        3.98623861e-093, 3.73165031e-103, 2.75132206e-113, 2.61367110e-129,
        3.60390711e-146]),
 'fe': array([4.49272849e-006, 1.41743511e-003, 8.67589108e-002, 8.52457843e-001,
        5.90680048e-002, 2.93136559e-004, 1.77275733e-007, 2.13558877e-014,
        1.22815025e-022, 1.80796581e-030, 9.03066470e-039, 2.55016369e-047,
        2.42432826e-056, 3.11895035e-065, 6.60868061e-074, 4.23884950e-082,
        1.42202017e-090

In [13]:
diskK.species_ion_abundances

{'h': array([3.88733750e-04, 9.99611266e-01]),
 'he': array([1.21486730e-03, 7.84902967e-07, 7.47150532e-01, 2.51633816e-01]),
 'c': array([2.72587553e-06, 3.56581725e-02, 8.12824374e-01, 1.48309174e-01,
        3.20554726e-03, 6.19587714e-09, 5.37465314e-15]),
 'ca': array([4.73372580e-007, 1.37998978e-003, 3.51136602e-001, 6.19509815e-001,
        2.79480848e-002, 2.50349206e-005, 4.45201594e-010, 2.44130466e-016,
        4.96806922e-023, 2.18098377e-029, 5.54077740e-036, 4.93120073e-042,
        4.72476209e-049, 6.25010645e-056, 2.25215779e-063, 4.37871230e-071,
        1.62266347e-079, 1.51375357e-088, 1.11220474e-097, 1.05289076e-112,
        1.44675516e-128]),
 'fe': array([3.06935824e-009, 9.65005197e-006, 5.88612700e-003, 5.76338349e-001,
        3.97966008e-001, 1.96812525e-002, 1.18609946e-004, 1.42389681e-010,
        8.16021236e-018, 1.19709672e-024, 5.95864919e-032, 1.67681530e-039,
        1.58853821e-047, 2.03659077e-055, 4.30030387e-063, 2.74866270e-070,
        9.18900